In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
%matplotlib inline

X, y = make_classification(n_samples=100000, n_features=2, n_redundant=0, n_repeated=0, 
                           n_informative=2, n_clusters_per_class=2, 
                           random_state=42) #for reproducibility 

In [ ]:
fig, ax = plt.subplots() 
fig.set_size_inches(10, 10)
plt.scatter(X[:,0], X[:,1], marker='.')
plt.axis('off')
plt.show()

In [ ]:
from tqdm import tqdm

def dist(x,y):
    return np.linalg.norm(x-y)

def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr == None:
        distances = [dist(v,p) for v in X]
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    return X, np.array(c)
    
def mdav(X, k):
    D = X
    clusters = []
    
    # Test feature. progress bar
    pbar = tqdm(total=len(D))
    
    while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v,xm) for v in D])
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = [dist(v,xr) for v in D]
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = dist_to_xr[:xsi]+dist_to_xr[xsi+1:]
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, c = cluster(D, xr, k, dist_to_xr)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
        #cluster of xs
        D, c = cluster(D, xs, k, None)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax([dist(v,xm) for v in D])
        D, xr = poprow(D, xri)
        #cluster of xr
        D, c = cluster(D, xr, k, None)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
        # rest of points
        clusters.append(D[:]) 
        
        # Test feature. progress bar
        pbar.update(len(D))
        
    else:
        # rest of points
        clusters.append(D[:])
        
        # Test feature. progress bar
        pbar.update(len(D))
    
    centroids = np.array([np.mean(c,axis=0) for c in clusters], copy=False)
    
    return clusters, centroids

def print_stats(clusters, centroids):
    ss = []
    for c,cen in zip(clusters, centroids):
        #cen = np.mean(c, axis=0)
        s = np.mean([dist(x,cen) for x in c], axis=0)
        ss.append(s)
        
    print(f'Number of clusters: {len(clusters)}')
    print(f'Mean of mean distances to centroids: {np.mean(ss, axis=0)}')

def plot_results(clusters):
    fig, ax = plt.subplots() 
    fig.set_size_inches(10, 10)
    for c, cen in zip(clusters, centroids):
        plt.scatter(c[:,0], c[:,1], marker='.', s=25)
        #plt.scatter(cen[0], cen[1], marker='o', s=100, edgecolor='k')
    plt.axis('off')
    plt.show()

In [ ]:
# Vanilla MDAV
k=5
clusters, centroids = mdav(X, k)
print_stats(clusters, centroids)
#plot_results(clusters)

# ALTERNATIVE IMPLEMENTATION

Substituting list comprehensions by NumPy automatic operand broadcast.

In [ ]:
from tqdm import tqdm

def dist(x,y):
    return np.linalg.norm(x-y, axis=1)

def poprow(arr,i):
    pop = arr[i]
    new_array = np.vstack((arr[:i],arr[i+1:]))
    return new_array,pop

def cluster(X, p, k, dist_to_xr):
    c = [p]
    
    if dist_to_xr is None:
        distances = dist(p,X)
    else:
        distances = dist_to_xr
        
    X = X[np.argpartition(distances, k-1)]
    c.extend(X[:k-1])
    X = X[k-1:]
    
    return X, np.array(c)
    
def mdav(X, k):
    D = X
    clusters = []
    
    # Test feature. progress bar
    pbar = tqdm(total=len(D))
    
    while len(D) >= 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax(dist(xm,D))
        D, xr = poprow(D, xri)
        # Furthest from furthest from centroid
        dist_to_xr = dist(xr,D)
        xsi = np.argmax(dist_to_xr)
        dist_to_xr = np.append(dist_to_xr[:xsi], dist_to_xr[xsi+1:], axis=0)
        D, xs = poprow(D, xsi) 

        #cluster of xr
        D, c = cluster(D, xr, k, dist_to_xr)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
        #cluster of xs
        D, c = cluster(D, xs, k, None)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
    if len(D) >= 2*k and len(D) < 3*k:
        # Centroid
        xm = np.mean(D, axis=0)
        # Furthest from centroid
        xri = np.argmax(dist(xm,D))
        D, xr = poprow(D, xri)
        #cluster of xr
        D, c = cluster(D, xr, k, None)
        clusters.append(c)
        
        # Test feature. progress bar
        pbar.update(k)
        
        # rest of points
        clusters.append(D[:]) 
        
        # Test feature. progress bar
        pbar.update(len(D))
        
    else:
        # rest of points
        clusters.append(D[:])
        
        # Test feature. progress bar
        pbar.update(len(D))
    
    centroids = np.array([np.mean(c,axis=0) for c in clusters], copy=False)
    
    return clusters, centroids

def print_stats(clusters, centroids):
    ss = []
    for c,cen in zip(clusters, centroids):
        #cen = np.mean(c, axis=0)
        s = np.mean(dist(cen,c), axis=0)
        ss.append(s)
        
    print(f'Number of clusters: {len(clusters)}')
    print(f'Mean of mean distances to centroids: {np.mean(ss, axis=0)}')

def plot_results(clusters):
    fig, ax = plt.subplots() 
    fig.set_size_inches(10, 10)
    for c, cen in zip(clusters, centroids):
        plt.scatter(c[:,0], c[:,1], marker='.', s=25)
        #plt.scatter(cen[0], cen[1], marker='o', s=100, edgecolor='k')
    plt.axis('off')
    plt.show()

In [ ]:
# Vectorized MDAV
k=5
clusters, centroids = mdav(X, k)
print_stats(clusters, centroids)
plot_results(clusters)

# STOP HERE

In [ ]:
k = 10000
c, cen = mdav(X, len(X) // 2)
c0, cen0 = mdav(c[0], k)
c1, cen1 = mdav(c[1], k)
clusters = np.vstack((c0,c1))
centroids = np.vstack((cen0,cen1))

print_stats(clusters, centroids)
plot_results(clusters)

In [ ]:
k = 10000
c, _ = mdav(X, len(X) // 2)
c0, _ = mdav(c[0], len(c[0]) // 2)
c1, _ = mdav(c[1], len(c[1]) // 2)
c00, cen00 = mdav(c0[0], k)
c01, cen01 = mdav(c0[1], k)
c10, cen10 = mdav(c1[0], k)
c11, cen11 = mdav(c1[1], k)

clusters = np.vstack((c00,c01,c10,c11))
centroids = np.vstack((cen00,cen01,cen10,cen11))

print_stats(clusters, centroids)
plot_results(clusters)

In [ ]:
k = 5
c, _ = mdav(X, len(X) // 2)
c0, _ = mdav(c[0], len(c[0]) // 2)
c1, _ = mdav(c[1], len(c[1]) // 2)
c00, _ = mdav(c0[0], len(c0[0]) // 2)
c01, _ = mdav(c0[1], len(c0[1]) // 2)
c10, _ = mdav(c1[0], len(c1[0]) // 2)
c11, _ = mdav(c1[1], len(c1[1]) // 2)

c000, cen000 = mdav(c00[0], k)
c001, cen001 = mdav(c00[1], k)
c010, cen010 = mdav(c01[0], k)
c011, cen011 = mdav(c01[1], k)
c100, cen100 = mdav(c10[0], k)
c101, cen101 = mdav(c10[1], k)
c110, cen110 = mdav(c11[0], k)
c111, cen111 = mdav(c11[1], k)

clusters = np.vstack(
    (
        c000,
        c001,
        c010,
        c011,
        c100,
        c101,
        c110,
        c111
    )
)

centroids = np.vstack(
    (
        cen000,
        cen001,
        cen010,
        cen011,
        cen100,
        cen101,
        cen110,
        cen111
    )
)

print_stats(clusters, centroids)
#plot_results(clusters)